In [1]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse

In [145]:
recipes_clean = pd.read_csv('/Users/giancarlotissot/code/mariliabreis/ingredient_matching/raw_data/recipes.csv')
ingredients_clean = pd.read_csv('/Users/giancarlotissot/code/mariliabreis/ingredient_matching/raw_data/ingredient_keys.csv')
ingredients_clean.drop(columns='Unnamed: 0',inplace=True)

In [149]:
df_pickle = pd.read_pickle('/Users/giancarlotissot/code/mariliabreis/ingredient_matching/raw_data/recipe_list/ingr_map.pkl')
ingredients_clean = df_pickle[['id','replaced','count','raw_ingr']]
# ingredients_clean.head(10)

In [5]:
new_df = recipes_clean[['id','ingredient_ids']].copy()
new_df = pd.DataFrame(new_df)

In [37]:
unique_ingredients_df = ingredients_clean.drop_duplicates().copy()
list_ingredient_ids = unique_ingredients_df['id'].sort_values()

final_df = pd.DataFrame(columns=list_ingredient_ids,index=new_df['id'],data=0)

In [40]:
%%time
reduced_df = new_df.head(50)
count =0
for id,recipe in new_df[['id','ingredient_ids']].to_numpy():
    count+=1
    print(f"looking at id: {count}",end="\r")
    columns = ast.literal_eval(recipe)
    final_df.loc[id,columns] = 1

CPU times: user 6min 41s, sys: 51.6 s, total: 7min 33s
Wall time: 10min 25s


In [44]:
%%time
final_df.to_csv(r'/Users/giancarlotissot/code/mariliabreis/ingredient_matching/raw_data/final_df.csv')

In [46]:
final_sparse = sparse.csr_matrix(final_df.values)

In [121]:
sparse.save_npz(r'/Users/giancarlotissot/code/mariliabreis/ingredient_matching/raw_data/sparse_final_df.npz',final_sparse)

In [148]:
### TO DO ###
### Use Sparse Matrix

## Input function

In [106]:
def get_id(ingredient_str):
    # Transforms string input to pre-processed ID
    ingredient_id = ingredients_clean[ingredients_clean['raw_ingr'] ==ingredient_str]
    ingredient_id.reset_index(inplace=True)
    return ingredient_id.loc[0,'id']

def get_name(ingredient_id):
    # Transforms ID back to pre-processed string
    ingredient_name = ingredients_clean[ingredients_clean['id'] == ingredient_id]
    ingredient_name.reset_index(inplace=True)
    return ingredient_name.loc[0,'replaced']

In [142]:
def find_match(ingredient_id,num_matches):
    # Returns list of ingredient IDs and count of occurances
    total = final_df[final_df[ingredient_id]==1].sum()
    ids = total.sort_values(ascending=False)
    return list(ids.head(num_matches).index)
    
def list_to_names(ingredient_id_list):
    # Takes in list of ingredient IDs and returns list of names
    list_ = []
    for id in ingredient_id_list:
        list_.append(get_name(id))
    return list_

def output_func(input_ingredient,num_matches=10):
    # Combines other functions into a workflow
    num_matches += 1
    result = get_id(input_ingredient)
    ids = find_match(result,num_matches)
    names = list_to_names(ids)
    names.pop(0)
    return names

## Processing Order
#get_id -> str to ID
#find_match -> Table
#list_to_names

In [144]:
output_func('romaine lettuce',15)

['tomato',
 'salt',
 'mayonnaise',
 'olive oil',
 'scallion',
 'onion',
 'garlic clove',
 'cheddar',
 'red onion',
 'tortilla',
 'sour cream',
 'dressing',
 'pepper',
 'sugar',
 'avocado']